# Sage notebook

In [1]:
from sage.algebras.lie_algebras.bch import bch_iterator
import numpy as np
import re
import time 

In [16]:
channels = 2  # d
depth = 3  # L

# Symbolic preprocessing 

In [17]:
Lbl = LieAlgebra(RR, channels, step=depth)
basislength = len(Lbl.gens())
print(basislength)

5


In [18]:
R = PolynomialRing(RR, 2*basislength, 'm')
m = R.gens()
print(R)
L = LieAlgebra(PolynomialRing(RR, 2*basislength, 'm'), channels, step=depth)
print(L)
L.inject_variables()

Multivariate Polynomial Ring in m0, m1, m2, m3, m4, m5, m6, m7, m8, m9 over Real Field with 53 bits of precision
Free Nilpotent Lie algebra on 5 generators (1.00000000000000*X_1, 1.00000000000000*X_2, 1.00000000000000*X_12, 1.00000000000000*X_112, 1.00000000000000*X_122) over Multivariate Polynomial Ring in m0, m1, m2, m3, m4, m5, m6, m7, m8, m9 over Real Field with 53 bits of precision
Defining X_1, X_2, X_12, X_112, X_122


In [20]:
Y = L.from_vector(vector(R, m[basislength:]))   # x^(j) (so be summed over)
Y

m5*X_1 + m6*X_2 + m7*X_12 + m8*X_112 + m9*X_122

In [21]:
X = L.from_vector(vector(R, m[:basislength]))  # y is inverse of mean (m symbols, to be computed)
X

m0*X_1 + m1*X_2 + m2*X_12 + m3*X_112 + m4*X_122

In [24]:
S = sum([Z for Z in bch_iterator(X, Y)])   # symbolic BCH formula 
S

(m0+m5)*X_1 + (m1+m6)*X_2 + (-0.500000000000000*m1*m5+0.500000000000000*m0*m6+m2+m7)*X_12 + (-0.0833333333333333*m0*m1*m5+0.0833333333333333*m1*m5^2+0.0833333333333333*m0^2*m6-0.0833333333333333*m0*m5*m6-0.500000000000000*m2*m5+0.500000000000000*m0*m7+m3+m8)*X_112 + (0.0833333333333333*m1^2*m5-0.0833333333333333*m0*m1*m6-0.0833333333333333*m1*m5*m6+0.0833333333333333*m0*m6^2+0.500000000000000*m2*m6-0.500000000000000*m1*m7+m4+m9)*X_122

In [25]:
Sdic = S.monomial_coefficients()       # apply dual basis (coefficients of lyndon basis elements)
Sdic

{(1,): m0 + m5,
 (2,): m1 + m6,
 (1, 2): -0.500000000000000*m1*m5 + 0.500000000000000*m0*m6 + m2 + m7,
 (1,
  1,
  2): -0.0833333333333333*m0*m1*m5 + 0.0833333333333333*m1*m5^2 + 0.0833333333333333*m0^2*m6 - 0.0833333333333333*m0*m5*m6 - 0.500000000000000*m2*m5 + 0.500000000000000*m0*m7 + m3 + m8,
 (1,
  2,
  2): 0.0833333333333333*m1^2*m5 - 0.0833333333333333*m0*m1*m6 - 0.0833333333333333*m1*m5*m6 + 0.0833333333333333*m0*m6^2 + 0.500000000000000*m2*m6 - 0.500000000000000*m1*m7 + m4 + m9}

In [9]:
Sval = list(Sdic.values())                # derive polynomials 
for i in range(len(Sval)):
    Sval[i] -= m[i]
Sval

[m829,
 m830,
 m831,
 m832,
 m833,
 -0.500000000000000*m1*m829 + 0.500000000000000*m0*m830 + m834,
 -0.500000000000000*m2*m829 + 0.500000000000000*m0*m831 + m835,
 -0.500000000000000*m3*m829 + 0.500000000000000*m0*m832 + m836,
 -0.500000000000000*m4*m829 + 0.500000000000000*m0*m833 + m837,
 -0.500000000000000*m2*m830 + 0.500000000000000*m1*m831 + m838,
 -0.500000000000000*m3*m830 + 0.500000000000000*m1*m832 + m839,
 -0.500000000000000*m4*m830 + 0.500000000000000*m1*m833 + m840,
 -0.500000000000000*m3*m831 + 0.500000000000000*m2*m832 + m841,
 -0.500000000000000*m4*m831 + 0.500000000000000*m2*m833 + m842,
 -0.500000000000000*m4*m832 + 0.500000000000000*m3*m833 + m843,
 -0.0833333333333333*m0*m1*m829 + 0.0833333333333333*m1*m829^2 + 0.0833333333333333*m0^2*m830 - 0.0833333333333333*m0*m829*m830 - 0.500000000000000*m5*m829 + 0.500000000000000*m0*m834 + m844,
 -0.0833333333333333*m0*m2*m829 + 0.0833333333333333*m2*m829^2 + 0.0833333333333333*m0^2*m831 - 0.0833333333333333*m0*m829*m831 - 0.5

In [10]:
def string_preproc(st_poly):
    result = re.sub(r'\^', '**',   st_poly)
    result = re.sub(r'()m([0-9]*)()', r'\1m[\2]\3', result)
    return "lambda m : "+result


In [11]:
#def string_readabilityL3d2(st_poly):
#    result = re.sub(r'm5', 'C_1',   st_poly)
#    result = re.sub(r'm6', 'C_2',   result)
#    result = re.sub(r'm7', 'C_3',   result)
#    result = re.sub(r'm8', 'C_4',   result)
#    result = re.sub(r'm9', 'C_5',   result)
#    result = re.sub(r'm0', 'M_1',   result)
#    result = re.sub(r'm1', 'M_2',   result)
#    result = re.sub(r'm2', 'M_3',   result)
#    result = re.sub(r'm3', 'M_4',   result)
#    result = re.sub(r'm4', 'M_5',   result)
#    result = re.sub('\*', ' ',   result)
#    return result
#outp_0 = [string_readabilityL3d2(str(a)) for a in Sval]
#outp_0

In [12]:
outp_0 = [string_preproc(str(a)) for a in Sval]
outp_0


['lambda m : m[829]',
 'lambda m : m[830]',
 'lambda m : m[831]',
 'lambda m : m[832]',
 'lambda m : m[833]',
 'lambda m : -0.500000000000000*m[1]*m[829] + 0.500000000000000*m[0]*m[830] + m[834]',
 'lambda m : -0.500000000000000*m[2]*m[829] + 0.500000000000000*m[0]*m[831] + m[835]',
 'lambda m : -0.500000000000000*m[3]*m[829] + 0.500000000000000*m[0]*m[832] + m[836]',
 'lambda m : -0.500000000000000*m[4]*m[829] + 0.500000000000000*m[0]*m[833] + m[837]',
 'lambda m : -0.500000000000000*m[2]*m[830] + 0.500000000000000*m[1]*m[831] + m[838]',
 'lambda m : -0.500000000000000*m[3]*m[830] + 0.500000000000000*m[1]*m[832] + m[839]',
 'lambda m : -0.500000000000000*m[4]*m[830] + 0.500000000000000*m[1]*m[833] + m[840]',
 'lambda m : -0.500000000000000*m[3]*m[831] + 0.500000000000000*m[2]*m[832] + m[841]',
 'lambda m : -0.500000000000000*m[4]*m[831] + 0.500000000000000*m[2]*m[833] + m[842]',
 'lambda m : -0.500000000000000*m[4]*m[832] + 0.500000000000000*m[3]*m[833] + m[843]',
 'lambda m : -0.0833

In [13]:
outp = [eval(a) for a in outp_0]
outp

[<function <lambda> at 0x16c1d6d40>,
 <function <lambda> at 0x16c1d67a0>,
 <function <lambda> at 0x16c1d6a70>,
 <function <lambda> at 0x16c1d6560>,
 <function <lambda> at 0x16c1d5ea0>,
 <function <lambda> at 0x16c1d65f0>,
 <function <lambda> at 0x16c1d64d0>,
 <function <lambda> at 0x16c1d6320>,
 <function <lambda> at 0x16c1d5f30>,
 <function <lambda> at 0x16c1d70a0>,
 <function <lambda> at 0x16c1d6cb0>,
 <function <lambda> at 0x16c1d5000>,
 <function <lambda> at 0x16c1d4ee0>,
 <function <lambda> at 0x16c1d4f70>,
 <function <lambda> at 0x16c1d5cf0>,
 <function <lambda> at 0x16c1d7400>,
 <function <lambda> at 0x16c1d75b0>,
 <function <lambda> at 0x16c1d76d0>,
 <function <lambda> at 0x16c1d77f0>,
 <function <lambda> at 0x16c1d7880>,
 <function <lambda> at 0x16c1d7910>,
 <function <lambda> at 0x16c1d79a0>,
 <function <lambda> at 0x16c1d7a30>,
 <function <lambda> at 0x16c1d7ac0>,
 <function <lambda> at 0x16c1d7b50>,
 <function <lambda> at 0x16c1d7be0>,
 <function <lambda> at 0x16c1d7c70>,
 

In [14]:
R

Multivariate Polynomial Ring in m0, m1, m2, m3, m4, m5, m6, m7, m8, m9, m10, m11, m12, m13, m14, m15, m16, m17, m18, m19, m20, m21, m22, m23, m24, m25, m26, m27, m28, m29, m30, m31, m32, m33, m34, m35, m36, m37, m38, m39, m40, m41, m42, m43, m44, m45, m46, m47, m48, m49, m50, m51, m52, m53, m54, m55, m56, m57, m58, m59, m60, m61, m62, m63, m64, m65, m66, m67, m68, m69, m70, m71, m72, m73, m74, m75, m76, m77, m78, m79, m80, m81, m82, m83, m84, m85, m86, m87, m88, m89, m90, m91, m92, m93, m94, m95, m96, m97, m98, m99, m100, m101, m102, m103, m104, m105, m106, m107, m108, m109, m110, m111, m112, m113, m114, m115, m116, m117, m118, m119, m120, m121, m122, m123, m124, m125, m126, m127, m128, m129, m130, m131, m132, m133, m134, m135, m136, m137, m138, m139, m140, m141, m142, m143, m144, m145, m146, m147, m148, m149, m150, m151, m152, m153, m154, m155, m156, m157, m158, m159, m160, m161, m162, m163, m164, m165, m166, m167, m168, m169, m170, m171, m172, m173, m174, m175, m176, m177, m178, m179

In [15]:
max([p.number_of_terms() for p in Sval])

99

# Loading the data

In [295]:
logSX = np.load('./data.npy')
N = logSX.shape[0]

N,B = logSX.shape
print(N,B)


10992 5


# Evaluation group mean

In [162]:
start_time = time.time()
res = np.zeros(basislength)
for k in range(basislength):
    res[k] = np.sum(outp[k](list(-res)+[logSX[:,_k] for _k in range(basislength) ]))/N
print(time.time() - start_time)
print(res)

0.005102872848510742
[ 0.03478179 -0.20569871 -0.00655954 -0.00063188  0.00107996]


# Experiement (left right)

In [163]:
logSXn = L.from_vector(vector(R, m[basislength:]))   # x^(j) (so be summed over)
y = -L.from_vector(vector(R, m[:basislength]))  # y is inverse of mean (m symbols, to be computed)
S = sum([Z for Z in bch_iterator(logSXn,y )])   # symbolic BCH formula 
Sdic = S.monomial_coefficients()       # apply dual basis (coefficients of lyndon basis elements)
Sval = list(Sdic.values())                # derive polynomials 
for i in range(len(Sval)):
    Sval[i] += m[i]
outp_0 = [string_preproc(str(a)) for a in Sval]
outp = [eval(a) for a in outp_0]
res = np.zeros(basislength)
for k in range(basislength):
    res[k] = np.sum(outp[k](list(res)+[logSX[:,_k] for _k in range(basislength) ]))/N
res

array([ 0.03478179, -0.20569871, -0.00655954, -0.00063188,  0.00107996])

In [177]:
np.ones(3)
R

Multivariate Polynomial Ring in x, y, z over Rational Field

In [181]:

#phi = R.hom(list(np.ones(10)))
R.<x,y,z> = PolynomialRing(QQ,3)
phi = R.hom(3*[1.])

In [182]:
phi(x+1+y)

3.00000000000000